<center>

# Discrete Event Systems

## Home Assignment 1

### Synchronization and Reachability Analysis

- - -

# Introduction

The objective of this home assignment is to get a deeper understanding of some basic features of discrete event systems. This will be achieved by implementing a few key functions in Python, including the computation of coreachable states, as well as the synchronous composition of two automata. These functions are then applied to the analysis of a simple coordination control system.

This home assignment is performed in *two member groups*. Write all your answers into this notebook and **submit only this notebook (.ipynb) on Canvas**

## Comments and Recommendations

* As before, the following resources will be of great help to you for this assignment:
 * DES lecture notes
 * [Python docs](https://docs.python.org/3/)
 * [Google](https://www.google.com)
 * [Wikipedia overview of syntax and semantics](https://en.wikipedia.org/wiki/Python_syntax_and_semantics) 
* We will work extensively with **sets** in this assignment: 
 * A set is created either through `set(*iterable*)` or by listing its elements between curly braces `{*iterable*}`; 
 * An empty set must be created by `set()`. `{}` will create an empty dictionary; 
 * You will also see often something like `{elem for elem in items if elem}`. This is called a *set comprehension* and builds efficiently a new set by iterating over `items` and adding the `elem` for which the if-statement is true; 
 * Have an extensive look at [the set documentation](https://docs.python.org/3/library/stdtypes.html#set) for more information on set methods. Use set methods as much as possible.  
* The assignment is written for Python 3.5 or later.
* If your code passes all the tests in this notebook, you are on a good way. That does not mean you have passed though! **We will test your code with additional edge cases.** So, convince yourself that everything is correct before you submit, e.g. *through your own test cases*.

- - -

# Data Structures

As in Assignment 0, we define the Transition and Automaton classes. They are structured the same as previously. We have included, however, a few validation methods that ensure that the Automaton is consistent at all time. You do not need to change anything here, but give it a thorough read anyway.  

In [ ]:
from collections import namedtuple

from util import plot_automaton


Transition = namedtuple(typename='Transition', field_names=[
                        'source', 'event', 'target'])


class Automaton(object):

    def __init__(self, states, init, events, trans, marked=None, forbidden=None):
        """
        This is the constructor of the automaton.

        At creation, the automaton gets the following attributes assigned:
        :param states: A set of states
        :param init: The initial state
        :param events: A set of events
        :param trans: A set of transitions
        :param marked: (Optional) A set of marked states
        :param forbidden: (Optional) A set of forbidden states
        """
        self.states = states
        self.init = init
        self.events = events
        self.trans = trans
        self.marked = marked if marked else set()
        self.forbidden = forbidden if forbidden else set()

    def __str__(self):
        """
        Prints the automaton in a pretty way.
        """
        return 'states: \n\t{}\n' \
               'init: \n\t{}\n' \
               'events: \n\t{}\n' \
               'transitions: \n\t{}\n' \
               'marked: \n\t{}\n' \
               'forbidden: \n\t{}\n'.format(
                   self.states, self.init, self.events,
                   '\n\t'.join([str(t) for t in self.trans]), self.marked, self.forbidden)

    def __setattr__(self, name, value):
        """Validates and protects the attributes of the automaton"""
        if name in ('states', 'events'):
            value = frozenset(self._validate_set(value))
        elif name == 'init':
            value = self._validate_init(value)
        elif name == 'trans':
            value = frozenset(self._validate_transitions(value))
        elif name in ('marked', 'forbidden'):
            value = frozenset(self._validate_subset(value))
        super(Automaton, self).__setattr__(name, value)

    def __getattribute__(self, name):
        """Returns a regular set of the accessed attribute"""
        if name in ('states', 'events', 'trans', 'marked', 'forbidden'):
            return set(super(Automaton, self).__getattribute__(name))
        else:
            return super(Automaton, self).__getattribute__(name)

    def __eq__(self, other):
        """Checks if two Automata are the same"""
        if type(other) is type(self):
            return self.__dict__ == other.__dict__
        return False

    @staticmethod
    def _validate_set(states):
        """Checks that states is a set and the states in it are strings or integers"""
        assert isinstance(states, set)
        for state in states:
            assert isinstance(state, str) or isinstance(
                state, int), 'A state must be either of type string or integer!'
        return states

    def _validate_subset(self, subset):
        """Validates the set and checks whether the states in the subset are part of the state set"""
        subset = self._validate_set(subset)
        assert subset.issubset(
            self.states), 'Marked and forbidden states must be subsets of all states!'
        return subset

    def _validate_init(self, state):
        """Checks whether the state is part of the state set"""
        assert isinstance(state, str) or isinstance(
            state, int), 'The initial state must be of type string or integer!'
        assert state in self.states, 'The initial state must be member of states!'
        return state

    def _validate_transitions(self, transitions):
        """Checks that all transition elements are part in the respective sets (states, events)"""
        assert isinstance(transitions, set)
        for transition in transitions:
            assert isinstance(transition, Transition)
            assert transition.source in self.states
            assert transition.event in self.events
            assert transition.target in self.states
        return transitions

- - -

# Task 1

## Task 1.1 - Reachability v.1 ***[1p]***

To start with, we revisit a few of the functions that you have implemented already in the previous assignment. Whereas Assignment 0 was intended to provide you with a test bench for programming in Python and developing algorithmic understanding, this task is to ensure that you got a correct implementation of `reach` for the rest of the assignment. You might be able to copy your previous solution here. 

A simple reachability algorithm based on set logic is as follows:

**Algorithm 1.** Reachability($\delta, Q_0$)

>**let** $k:=0$
>
>**repeat**
>
>>$k := k+1$
>>
>>$Q_k := Q_{k-1} \cup \{q^+ \mid \exists q \in Q_{k-1} \wedge \delta(q,\sigma) = q^+ \}$
>>
>**until** $Q_k = Q_{k-1}$
>
**return** $Q_k$

This is a so called fix point operation. We want you to implement a version of this algorithm. Check the lecture notes for further explanation. Other implementations, that use e.g. a recursion, will not be accepted even if correct.


Here are some helper functions that might help you with your implementation of the algorithm in code.


In [ ]:
def filter_trans_by_source(trans, states_to_keep):
    """Returns a new set containing all transitions where the source is in states_to_keep"""
    return {t for t in trans if t.source in states_to_keep}

def filter_trans_by_events(trans, events_to_keep):
    """Returns a new set containing all transitions where the event is in events_to_keep"""
    return {t for t in trans if t.event in events_to_keep}

def filter_trans_by_target(trans, states_to_keep):
    """Returns a new set containing all transitions where the target is in states_to_keep"""
    return {t for t in trans if t.target in states_to_keep}

def extract_elems_from_trans(trans, field):
    """ 
    Returns a new set with just the elements in a field of all transitions. 
    E.g. field='source' for all source states 
    or field='event' or field='target'
    """
    return {getattr(t, field) for t in trans}

def flip_trans(trans):
    """ Flips the direction of the transitions in the set"""
    return {Transition(t.target, t.event, t.source) for t in trans}

In [ ]:
def reach(trans, start_state):
    """
    Returns the forward reachable states of a transition set
    
    :param trans: set of transitions
    :param start_state: set of states
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    reachable_states = set(start_state)
    states_to_explore = set(start_state)

    while states_to_explore:
        current_state = states_to_explore.pop()

        for transition in trans:
            if transition.source == current_state and transition.target not in reachable_states:
                reachable_states.add(transition.target)
                states_to_explore.add(transition.target)
    
    return reachable_states

As before, the below is a test cell. You can execute it to test your function with different inputs. The **assert** statements make sure that your function returns the expected results. We encourage you to add also your own tests.

In [ ]:
# Space for your own tests

In [ ]:
# Single state
assert reach(set(), {1}) == {1}
assert reach({Transition(1, 'a', 1)}, {1}) == {1}

# Simple automata
assert reach({Transition(1, 'a', 2)}, {1}) == {1, 2}
assert reach({Transition(2, 'a', 1)}, {1}) == {1}
assert reach({Transition(2, 'a', 1), Transition(1, 'a', 2)}, {1}) == {1, 2}

# Parallel transitions
assert reach({Transition(1, 'a', 2), Transition(1, 'b', 2)}, {1}) == {1, 2}
assert reach({Transition(2, 'a', 1), Transition(2, 'b', 1)}, {1}) == {1}

# Larger automata
assert reach({Transition(1, 'a', 2), Transition(2, 'a', 3),
              Transition(3, 'a', 1)}, {1}) == {1, 2, 3}

## Task 1.2 - Coreachability ***[1p]***

Similarly, the set of backwards reachable states is defined as:

**Algorithm 2.** BackwardReachability($\delta, Q_0$)

>**let** $k:=0$
>
>**repeat**
>
>>$k := k+1$
>>
>>$Q_k := Q_{k-1} \cup \{q \mid \delta(q,\sigma) \in Q_{k-1} \}$
>>
>**until** $Q_k = Q_{k-1}$
>
**return** $Q_k$

* Implement Algorithm 2.

In [ ]:
def coreach(trans, start_state):
    """
    Returns the coreachable (backward reachable) states of a transition set
    
    :param trans: set of transitions
    :param start_state: set of states
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # return coreach_states    

    coreach_states  = set(start_state)
    states_to_explore = set(start_state)

    while states_to_explore:
        current_state = states_to_explore.pop()

        for currnet_transition in trans:
            if currnet_transition.target == current_state and currnet_transition.source not in coreach_states:
                coreach_states.add(currnet_transition.source)
                states_to_explore.add(currnet_transition.source)

    return coreach_states 

In [ ]:
# Space for your own tests

In [ ]:
# Single state
assert coreach(set(), {1}) == {1}
assert coreach({Transition(1, 'a', 1)}, {1}) == {1}

# Simple automata
assert coreach({Transition(1, 'a', 2)}, {1}) == {1}
assert coreach({Transition(2, 'a', 1)}, {1}) == {1, 2}
assert coreach({Transition(2, 'a', 1), Transition(1, 'a', 2)}, {1}) == {1, 2}

# Parallel transitions
assert coreach({Transition(1, 'a', 2), Transition(1, 'b', 2)}, {1}) == {1}
assert coreach({Transition(2, 'a', 1), Transition(2, 'b', 1)}, {1}) == {1, 2}

# Larger automata
assert coreach({Transition(1, 'a', 2), Transition(2, 'a', 3),
                Transition(3, 'a', 1)}, {1}) == {1, 2, 3}

## Task 1.3 - Synchronous composition ***[1p]***
* Implement the synchronous composition of two automaton. Exclude all unreachble states in the result.

* *Note:* The synchronization function can be implemented in different ways. We suggest to loop through all events, taking common and individual events for the two automata into account. In this event loop appropriate states are merged together according to the basic definition. A number of unreachable states will then be included.   
* *Note:* The synchronized function shall include the complete automaton, including any marked and forbidden states of the synchronized automaton.

Here are again some helper functions that might help you with your implementation of the algorithm in code.

In [ ]:
def merge_label(label1, label2):
    """Creates a new label based on two labels"""
    return '{}.{}'.format(label1, label2)

def cross_product(setA, setB):
    """Computes the crossproduct of two sets"""
    return {merge_label(a, b) for b in setB for a in setA}

In [ ]:
def synch(aut1, aut2):
    """
    Returns the synchronous composition of two automata.
    
    :param aut1: Automaton
    :param aut2: Automaton
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return aut1aut2

# Test the synch function with 2 automata
b1 = Automaton(states={'a', 'b'},
               init='a',
               events={1},
               trans={Transition('a', 1, 'b')},
               marked={'b'})
b2 = Automaton(states={'c', 'd', 'e'},
               init='c',
               events={1, 2},
               trans={Transition('c', 1, 'd'), Transition('d', 2, 'e'), Transition('e', 1, 'c')})
b1b2 = synch(b1, b2)

In [ ]:
# Space for your own tests

In [ ]:
# Two 1-state automata
a1 = Automaton(states={1},
               init=1,
               events=set(),
               trans=set())
a2 = Automaton(states={2},
               init=2,
               events=set(),
               trans=set())
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.2'}  # TC - 1.1
assert a1a2.init == '1.2'  # TC - 1.2
assert a1a2.events == set()  # TC - 1.3
assert a1a2.trans == set()  # TC - 1.4

a1 = Automaton(states={1},
               init=1,
               events=set(),
               trans=set(),
               marked={1})
a2 = Automaton(states={2},
               init=2,
               events=set(),
               trans=set())
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.2'}  # TC - 2.1
assert a1a2.init == '1.2'  # TC - 2.2
assert a1a2.events == set()  # TC - 2.3
assert a1a2.trans == set()  # TC - 2.4
assert a1a2.marked == {'1.2'}  # TC - 2.5

a1 = Automaton(states={1},
               init=1,
               events=set(),
               trans=set(),
               forbidden={1})
a2 = Automaton(states={2},
               init=2,
               events=set(),
               trans=set())
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.2'}  # TC - 3.1
assert a1a2.init == '1.2'  # TC - 3.2
assert a1a2.events == set()  # TC - 3.3
assert a1a2.trans == set()  # TC - 3.4
assert a1a2.forbidden == {'1.2'}  # TC - 3.5

# 2-state automaton and 1-state automaton
a1 = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)},
               marked={2})
a2 = Automaton(states={3},
               init=3,
               events=set(),
               trans=set())
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.3'}  # TC - 4.1
assert a1a2.init == '1.3'  # TC - 4.2
assert a1a2.events == {'a'}  # TC - 4.3
assert a1a2.trans == {Transition('1.3', 'a', '2.3')}  # TC - 4.4
assert a1a2.marked == {'2.3'}  # TC - 4.5

a1 = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)},
               forbidden={2})
a2 = Automaton(states={3},
               init=3,
               events=set(),
               trans=set())
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.3'}  # TC - 5.1
assert a1a2.init == '1.3'  # TC - 5.2
assert a1a2.events == {'a'}  # TC - 5.3
assert a1a2.trans == {Transition('1.3', 'a', '2.3')}  # TC - 5.5
assert a1a2.forbidden == {'2.3'}  # TC - 5.6

# Two 2-state automata
a1 = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)},
               marked={2})
a2 = Automaton(states={3, 4},
               init=3,
               events={'b'},
               trans={Transition(3, 'b', 4)})
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.3', '1.4', '2.4'}  # TC - 6.1
assert a1a2.init == '1.3'  # TC - 6.2
assert a1a2.events == {'a', 'b'}  # TC - 6.3
assert a1a2.trans == {Transition('1.3', 'a', '2.3'), Transition(
    '1.4', 'a', '2.4'), Transition('1.3', 'b', '1.4'), Transition('2.3', 'b', '2.4')}  # TC -6.4
assert a1a2.marked == {'2.3', '2.4'}  # TC - 6.5

a1 = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)},
               marked={2})
a2 = Automaton(states={3, 4},
               init=3,
               events={'a'},
               trans={Transition(3, 'a', 4)})
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.4'}  # TC - 7.1
assert a1a2.init == '1.3'  # TC - 7.2
assert a1a2.events == {'a'}  # TC - 7.3
assert a1a2.trans == {Transition('1.3', 'a', '2.4')}  # TC - 7.4
assert a1a2.marked == {'2.4'}  # TC - 7.5

a1 = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)},
               forbidden={2})
a2 = Automaton(states={3, 4},
               init=3,
               events={'b'},
               trans={Transition(3, 'b', 4)},
               forbidden={4})
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.3', '1.4', '2.4'}  # TC - 8.1
assert a1a2.init == '1.3'  # TC - 8.2
assert a1a2.events == {'a', 'b'}  # TC - 8.3
assert a1a2.trans == {Transition('1.3', 'a', '2.3'), Transition(
    '1.4', 'a', '2.4'), Transition('1.3', 'b', '1.4'), Transition('2.3', 'b', '2.4')}  # TC - 8.4
assert a1a2.forbidden == {'2.3', '1.4', '2.4'}  # TC - 8.5

a1 = Automaton(states={1, 2},
               init=1,
               events={'a', 'b'},
               trans={Transition(1, 'a', 2)},
               marked={2})
a2 = Automaton(states={3, 4},
               init=3,
               events={'b'},
               trans={Transition(3, 'b', 4)})
a1a2 = synch(a1, a2)
assert a1a2.states == {'1.3', '2.3'}  # TC - 9.1
assert a1a2.init == '1.3'  # TC - 9.2
assert a1a2.events == {'a', 'b'}  # TC - 9.3
assert a1a2.trans == {Transition('1.3', 'a', '2.3')}  # TC - 9.4
assert a1a2.marked == {'2.3'}  # TC - 9.5

# Different symbols
b1 = Automaton(states={'a', 'b'},
               init='a',
               events={1, 2},
               trans={Transition('a', 1, 'b')},
               marked={'b'})
b2 = Automaton(states={'c', 'd'},
               init='c',
               events={2},
               trans={Transition('c', 2, 'd')})
b1b2 = synch(b1, b2)
assert b1b2.states == {
    'a.c', 'b.c'}, 'Different symbols should not cause error!'
assert b1b2.init == 'a.c', 'Different symbols should not cause error!'
assert b1b2.events == {1, 2}, 'Different symbols should not cause error!'
assert b1b2.trans == {Transition(
    'a.c', 1, 'b.c')}, 'Different symbols should not cause error!'
assert b1b2.marked == {'b.c'}, 'Different symbols should not cause error!'

- - -

# Task 2

Consider the following two plant models given by the formal languages

<center>
    $L(P_1) = \overline{(ab)^*} \\
    L(P_2) = \overline{(c(d+e))^*}$

and the marked specification languages 

<center>
    $L_m(Sp_1) = (bc)^* \\
    L_m(Sp_2) = (cd)^*$
    
and the synchronised system

<center>
    $S = P_1 \parallel P_2 \parallel Sp_1 \parallel Sp_2$
    
The plant models are already given as automaton ***p1*** and ***p2*** below.

In [ ]:
p1 = Automaton(states={'p11', 'p12'},
               init='p11',
               events={'a', 'b'},
               trans={Transition('p11', 'a', 'p12'),
                      Transition('p12', 'b', 'p11')})
p2 = Automaton(states={'p21', 'p22'},
               init='p21',
               events={'c', 'd', 'e'},
               trans={Transition('p21', 'c', 'p22'),
                      Transition('p22', 'd', 'p21'),
                      Transition('p22', 'e', 'p21')})

## Task 2.1 - Petri Net

* Formulate a Petri net for the synchronized system *S* ***[1p]*** 
* and generate the corresponding reachability graph. ***[1p]***
* *Note:* It might be easier to first draw those with a pen on paper and then transfer it into the notebook.

In [ ]:
from util import plot_petrinet

PetriNet = namedtuple(typename='PetriNet', field_names=['places', 'transitions', 'arcs'])
Place = namedtuple('Place', ['label', 'marking'])
Arc = namedtuple('Arc', ['source', 'target', 'weight'])

# You can use the plotting function to plot your petri net for S
example = PetriNet(places={Place('p1', 1), Place('p2', 0)},
                   transitions={'t1'},
                   arcs={Arc('p1', 't1', 1), Arc('t1', 'p2', 1)})
plot_petrinet(example, 'example')

In [ ]:
def make_synchronized_petri_net_S():
    """
    A little helper function to create the Petri net of S.
    
    You can just hard code the elements of the net just like in the example above.
    We will call this function later in the tests.
    """
    # YOUR CODE HERE
    
    S = PetriNet(places={Place('p1', 4), Place('p2', 0)},
                   transitions={'t1','t2'},
                   arcs={Arc('p1', 't1', 1), Arc('t1', 'p2', 1), Arc('p2', 't2', 1),Arc('t2', 'p1', 1)})
    # raise NotImplementedError()
    return S

plot_petrinet(make_synchronized_petri_net_S(), 'S_petrinet')

In [ ]:
S = make_synchronized_petri_net_S()
assert isinstance(S, PetriNet)
assert isinstance(S.places.pop(), Place)
assert isinstance(S.arcs.pop(), Arc)

Similarly, we create the reachability graph.

In [ ]:
from util import plot_digraph

DiGraph = namedtuple(typename='DiGraph', field_names=['nodes', 'init', 'edges'])
Edge = namedtuple('Edge', ['source', 'target', 'label'])

# You can use the plotting function to plot your reachability graph for S
example = DiGraph(nodes={'01', '10'},
                  init='01',
                  edges={Edge('01', '10', 'a')})
plot_digraph(example, 'example')

In [ ]:
def make_reachability_graph():
    """
    A little helper function to create the reachability graph of S.
    
    You can just hard code the elements of the graph just like in the example above.
    We will call this function later in the tests.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return G

plot_digraph(make_reachability_graph(), 'S_reach_graph')

In [ ]:
G = make_reachability_graph()
assert isinstance(G, DiGraph)
assert isinstance(G.nodes, set)
assert all([n.isnumeric() for n in G.nodes])
assert isinstance(G.edges.pop(), Edge)

## Task 2.2 - Synchronized Automaton

* Generate the automaton for the synchronized system *S* by the implemented functions `synch`, ***[1p]***
* and compare the results with the reachability graph in **2.1**.

In [ ]:
def make_synchronized_automaton_S():
    """
    A little helper function to create the automaton of S.

    Define the automata sp1 and sp2 in here too.
    We will call this function later in the tests.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return S

plot_automaton(make_synchronized_automaton_S(), 'P1P2SP1SP2')

In [ ]:
S = make_synchronized_automaton_S()
assert S.init == 'p11.p21.sp11.sp21', 'Please, follow the synchronization order specified in Task 2!'

* Compare the synchronized automaton of *S* to the reachability graph. Write down your conclusions in the cell below. ***[1p]***

YOUR ANSWER HERE

## Task 2.3 - Blocking states

* Calculate the reachable and coreachable states in *S* by the implemented functions `reach` and `coreach`, and identify blocking states (states from where no marked state can be reached). ***[1p]***

In [ ]:
reachable = None
coreachable = None 
blocking = None
# YOUR CODE HERE
raise NotImplementedError()

print('The blocking states are: ', blocking)

In [ ]:
assert reachable is not None
assert coreachable is not None
assert blocking is not None

## Task 2.4 - Modified Spec

* Modify the specifications to avoid the blocking states, and verify that no blocking states are achieved by showing that the reachable and coreachable states of *S* are the same for the modified specifications. ***[1p]***
* Explain your modifications first in natural language. ***[1p]***

YOUR ANSWER HERE

In [ ]:
sp1_new = None

# YOUR CODE HERE
raise NotImplementedError()

plot_automaton(sp1_new, 'SP1_new') 

In [ ]:
sp2_new = None

# YOUR CODE HERE
raise NotImplementedError()

plot_automaton(sp2_new, 'SP2_new') 

In [ ]:
# Now synchronize with the new specifications
S_new = synch(synch(p1, p2), synch(sp1_new, sp2_new))

reachable_new = None
coreachable_new = None 
blocking_new = None
# YOUR CODE HERE
raise NotImplementedError()

print('The blocking states now are: ', blocking_new)
plot_automaton(S_new, 'S_new') 

In [ ]:
assert reachable_new is not None
assert coreachable_new is not None
assert blocking_new is not None


- - -

Congratulations! You have reached the end of the assignment. Now,

* Save the notebook
* Upload the .ipynb file to Canvas
* Enjoy the rest of the day! 